In [ ]:
# Import necessary libraries for environment, agents, HTTP requests, data modeling, and display
import os
import asyncio
import requests
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from typing import Dict
from IPython.display import display, Markdown

# Load environment variables from .env file
# override=True ensures existing variables are updated if present
load_dotenv(override=True)

# Retrieve Discord webhook URL from environment variables
DISCORD_WEBHOOK_URL = os.getenv('DISCORD_WEBHOOK_URL')
if not DISCORD_WEBHOOK_URL:
    raise ValueError("DISCORD_WEBHOOK_URL environment variable not set. Please add it to your .env file.")

# Define constant for number of searches to perform
HOW_MANY_SEARCHES = 3

print("Imports and initial setup complete.")

In [ ]:
# Define instructions for the search agent to summarize web results
SEARCH_AGENT_INSTRUCTIONS = """You are a research assistant. Given a search term, you search the web for that term and produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300 words. Capture the main points. Write succinctly, no need to have complete sentences or good grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the essence and ignore any fluff. Do not include any additional commentary other than the summary itself."""

# Define instructions for the planner agent to generate search terms
PLANNER_AGENT_INSTRUCTIONS = f"""You are a helpful research assistant. Given a query, come up with a set of web searches to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."""

# Define instructions for the writer agent to create a detailed report
WRITER_AGENT_INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

# Define instructions for the Discord agent to send formatted messages
DISCORD_AGENT_INSTRUCTIONS = """You are able to send a nicely formatted markdown message to a Discord channel based on a detailed report.\nYou will be provided with a detailed report. You should use your tool to send one message, providing the report converted into clean, well presented markdown with an appropriate subject line for the Discord message."""

print("Agent instructions defined.")

In [ ]:
# Define a Pydantic model for a single web search item
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

# Define a Pydantic model for the web search plan
class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

# Define a Pydantic model for the report output
class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report")
    follow_up_questions: list[str] = Field(description="Suggested topics to research further")

print("Pydantic models for structured outputs defined.")

In [ ]:
# Create a search agent to perform web searches and summarize results
search_agent = Agent(
    name="Search agent",
    instructions=SEARCH_AGENT_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

# Create a planner agent to generate a structured web search plan
planner_agent = Agent(
    name="PlannerAgent",
    instructions=PLANNER_AGENT_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

# Create a writer agent to produce a detailed report
writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_AGENT_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

print("Search, Planner, and Writer agents defined.")

In [11]:
# Define a tool to send messages to Discord via webhook with embedded content
@function_tool
def send_discord_message(subject: str, content: str) -> Dict[str, str]:
    """
    Sends a message to a Discord channel via webhook with the given subject and content.
    Handles long content by splitting it into multiple embeds.
    """
    if not DISCORD_WEBHOOK_URL:
        return {"status": "error", "message": "Discord Webhook URL is not set."}

    # Define Discord embed limits
    MAX_DESCRIPTION_LENGTH = 4096
    MAX_EMBEDS_PER_MESSAGE = 10
    MAX_TOTAL_EMBED_CHARS = 6000

    # Split content into chunks that fit Discord's embed description limit
    content_chunks = []
    current_chunk = ""
    for line in content.splitlines(True):
        if len(current_chunk) + len(line) <= MAX_DESCRIPTION_LENGTH:
            current_chunk += line
        else:
            content_chunks.append(current_chunk)
            current_chunk = line
    if current_chunk:
        content_chunks.append(current_chunk)

    # Create embeds for each chunk
    embeds = []
    for i, chunk in enumerate(content_chunks):
        if len(embeds) >= MAX_EMBEDS_PER_MESSAGE:
            break

        embed = {
            "title": f"{subject} (Part {i+1})" if i > 0 else subject,
            "description": chunk,
            "color": 3447003
        }
        embeds.append(embed)

    # Check if total embed characters exceed Discord's limit
    total_chars = sum(len(embed['description']) for embed in embeds) + sum(len(embed['title']) for embed in embeds)
    if total_chars > MAX_TOTAL_EMBED_CHARS:
        print(f"Warning: Total embed characters ({total_chars}) exceed Discord's 6000 limit. Truncating report for Discord.")
        embeds = []
        current_total_chars = 0
        for i, chunk in enumerate(content_chunks):
            title = f"{subject} (Part {i+1})" if i > 0 else subject
            if current_total_chars + len(chunk) + len(title) <= MAX_TOTAL_EMBED_CHARS:
                embeds.append({
                    "title": title,
                    "description": chunk,
                    "color": 3447003
                })
                current_total_chars += len(chunk) + len(title)
            else:
                break
        if not embeds and content_chunks:
            embeds.append({
                "title": subject,
                "description": content_chunks[0][:MAX_DESCRIPTION_LENGTH],
                "color": 3447003
            })

    # Prepare and send the payload to Discord
    payload = {"embeds": embeds}
    try:
        response = requests.post(DISCORD_WEBHOOK_URL, json=payload)
        response.raise_for_status()
        return {"status": "success", "message": "Discord message sent successfully."}
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": f"Failed to send Discord message: {e}. Response: {response.text if 'response' in locals() else 'No response'}"}

In [ ]:
# Create a Discord agent to send formatted messages using the webhook tool
discord_agent = Agent(
    name="Discord Agent",
    instructions=DISCORD_AGENT_INSTRUCTIONS,
    tools=[send_discord_message],
    model="gpt-4o-mini",
)

print("Discord Agent defined.")

In [ ]:
# Define a function to plan web searches for a query
async def plan_searches(query: str):
    """Use the planner_agent to plan which searches to run for the query"""
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

# Define a function to perform searches concurrently
async def perform_searches(search_plan: WebSearchPlan):
    """Call search() for each item in the search plan concurrently"""
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

# Define a function to execute a single search
async def search(item: WebSearchItem):
    """Use the search agent to run a web search for each item in the search plan"""
    input_message = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input_message)
    return result.final_output

# Define a function to write a report based on search results
async def write_report(query: str, search_results: list[str]):
    """Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input_message = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input_message)
    print("Finished writing report")
    return result.final_output

print("Orchestration functions for planning, searching, and writing report defined.")

In [ ]:
# Define a function to send a report to Discord
async def send_report_to_discord(report: ReportData):
    """Use the discord agent to send a Discord message with the report"""
    print("Preparing Discord message...")
    result = await Runner.run(discord_agent, report.markdown_report)
    print(f"Discord message status: {result.final_output}")
    return report

print("Orchestration function for sending report to Discord defined.")

In [ ]:
# Define the research query
query = "The impact of quantum computing on cryptography by 2030"

print(f"Starting research for query: \"{query}\"")

# Execute the research workflow
with trace("Research trace"):
    print("Starting research process...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_report_to_discord(report)
    print("Hooray! Research and Discord notification complete.")

# Display the final report in the notebook
display(Markdown(report.markdown_report))
print(f"Short Summary: {report.short_summary}")
print(f"Follow-up Questions: {report.follow_up_questions}")

print("\nWorkflow execution complete. Check your Discord channel for the report!")
print("Remember to check the trace at https://platform.openai.com/traces for detailed execution steps.")